In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import numpy as np
import missingno as ms
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from cuml.svm import SVC
from cuml.ensemble import RandomForestClassifier as cuRFC
from cuml.linear_model import LogisticRegression
import tensorflow as tf
from cuml import LinearRegression
from cuml.linear_model import LinearRegression

In [3]:
df = pd.read_csv('../input/ucl-ai-society-football-match-prediction/Kaggle Data/train.csv')
df_test = pd.read_csv('../input/ucl-ai-society-football-match-prediction/Kaggle Data/test.csv')
df

In [4]:
ms.matrix(df)

In [5]:
df['home_adv'] = df['home_team']==df['country']
df['home_disadv'] = df['away_team']==df['country']
df_test['home_adv'] = df_test['home_team']==df_test['country']
df_test['home_disadv'] = df_test['away_team']==df_test['country']

In [6]:
teams = {}
booleanDict = {
    False : 0,
    True : 1
}
counter = 1
for team in df['home_team'] :
    if team not in teams :
        teams[team] = counter
        counter += 1
for team in df['away_team'] :
    if team not in teams :
        teams[team] = counter
        counter+=1
for i in range(len(df['home_team'])) :
    df['home_team'][i] = teams[df['home_team'][i]]
for i in range(len(df['away_team'])) :
    df['away_team'][i] = teams[df['away_team'][i]]
for i in range(len(df_test['home_team'])) :
    df_test['home_team'][i] = teams[df_test['home_team'][i]]
for i in range(len(df_test['away_team'])) :
    df_test['away_team'][i] = teams[df_test['away_team'][i]]
for i in range(len(df['neutral'])) :
    df['neutral'][i] = booleanDict[df['neutral'][i]]
    df['home_adv'][i] = booleanDict[df['home_adv'][i]]
    df['home_disadv'][i] = booleanDict[df['home_disadv'][i]]
for i in range(len(df_test['neutral'])) :
    df_test['neutral'][i] = booleanDict[df_test['neutral'][i]]
    df_test['home_adv'][i] = booleanDict[df_test['home_adv'][i]]
    df_test['home_disadv'][i] = booleanDict[df_test['home_disadv'][i]]

In [7]:
df = df.drop(['country'],axis=1)
df_test = df_test.drop(['country'],axis=1)

In [8]:
df

In [9]:
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(df.drop(['Outcome'],axis=1), df['Outcome'])
print(df.columns)
clf.feature_importances_

In [10]:
train_x,test_x,train_y,test_y = train_test_split(df.drop(['Outcome','id','home_disadv'],axis=1),df['Outcome'],test_size=0.2,random_state=0)

In [11]:

clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(train_x, train_y)
print(train_x.columns)
clf.feature_importances_

In [12]:
train_x

In [13]:
X = np.asarray(train_x.values, dtype=np.float32)
y = np.asarray(train_y.values, dtype=np.float32)
lr = LinearRegression(fit_intercept = False, normalize = False,algorithm = "svd")
reg = lr.fit(X,y)
preds = reg.predict(np.asarray(test_x.values, dtype=np.float32))
newPreds = []
for i in range(len(preds)) :
    if preds[i] <= 0.5 :
        newPreds.append(0)
    else :
        newPreds.append(1)
preds = reg.predict(np.asarray(test_x.values, dtype=np.float32))
from cuml.metrics.accuracy import accuracy_score
print(accuracy_score(test_y,np.asarray(newPreds)))
import cuml
MAE = cuml.metrics.regression.mean_absolute_error(test_y,np.asarray(newPreds))
print(MAE)

In [14]:
X = np.asarray(train_x.values, dtype=np.float32)
y = np.asarray(train_y.values, dtype=np.float32)
clf = SVC(kernel='rbf', degree=3, gamma='auto', C=1)
clf.fit(X, y)
cuml_predict = clf.predict(np.asarray(test_x.values, dtype=np.float32))
from cuml.metrics.accuracy import accuracy_score
print(accuracy_score(test_y,cuml_predict))

In [15]:
X = np.asarray(train_x.values, dtype=np.float32)
y = np.asarray(train_y.values, dtype=np.float32)
cuml_model = cuRFC(max_features=1.0,
                   n_bins=128,
                   n_estimators=200)
cuml_model.fit(X,y)
cuml_predict = cuml_model.predict(np.asarray(test_x.values, dtype=np.float32))
from cuml.metrics.accuracy import accuracy_score
print(accuracy_score(test_y,cuml_predict))

In [16]:
X = np.asarray(train_x.values, dtype=np.float32)
y = np.asarray(train_y.values, dtype=np.float32)
reg = LogisticRegression()
reg.fit(X,y)
preds = reg.predict(np.asarray(test_x.values, dtype=np.float32))
from cuml.metrics.accuracy import accuracy_score
print(accuracy_score(test_y,preds))

In [17]:
train_x,test_x,train_y,test_y = train_test_split(df.drop(['Outcome','id','home_disadv'],axis=1),df['Outcome'],test_size=0.2,random_state=0)

In [18]:
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units = 500,activation='relu'))
ann.add(tf.keras.layers.Dense(units = 500,activation='relu'))
ann.add(tf.keras.layers.Dense(units = 500,activation='relu'))
ann.add(tf.keras.layers.Dense(units = 1,activation='sigmoid'))
ann.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
history = ann.fit(train_x.astype(float),train_y.astype(float),batch_size=32,epochs=2)

In [19]:
loss=history.history['loss']
acc=history.history['accuracy']
epochs=range(len(loss))
import matplotlib.pyplot as plt
plt.plot(epochs,loss)
plt.plot(epochs,acc)
plt.show()
prob = ann.predict(test_x.astype(float))
preds = []
for prediction in prob :
    if prediction<=0.5 :
        preds.append(0)
    else :
        preds.append(1)
import cuml
MAE = cuml.metrics.regression.mean_absolute_error(test_y,np.asarray(preds))
print(MAE)

In [20]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units = 100,activation='relu'))
ann.add(tf.keras.layers.Dropout(.4))
ann.add(tf.keras.layers.Dense(units = 100,activation='relu'))
ann.add(tf.keras.layers.Dropout(.4))
ann.add(tf.keras.layers.Dense(units = 100,activation='relu'))
ann.add(tf.keras.layers.Dropout(.4))
ann.add(tf.keras.layers.Dense(units = 1,activation='sigmoid'))
ann.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])
history=ann.fit(train_x.astype(float),train_y.astype(float),batch_size=32,epochs=2)

In [21]:
loss=history.history['loss']
acc=history.history['accuracy']
epochs=range(len(loss))
import matplotlib.pyplot as plt
plt.plot(epochs,loss)
plt.plot(epochs,acc)
plt.show()

In [22]:
prob = ann.predict(test_x.astype(float))
preds = []
for prediction in prob :
    if prediction<=0.5 :
        preds.append(0)
    else :
        preds.append(1)
import cuml
MAE = cuml.metrics.regression.mean_absolute_error(test_y,np.asarray(preds))
print(MAE)

In [23]:
df_test = df_test.drop(['id','home_disadv'],axis=1)
df_test

In [24]:
X = np.asarray(train_x.values, dtype=np.float32)
y = np.asarray(train_y.values, dtype=np.float32)
cuml_model = cuRFC(max_features=1.0,
                   n_bins=128,
                   n_estimators=200)
cuml_model.fit(X,y)
prob = cuml_model.predict(np.asarray(df_test.values, dtype=np.float32))
preds = []
for prediction in prob :
    if prediction<=0.5 :
        preds.append(0)
    else :
        preds.append(1)
counter = 30000
ids = []
for i in range(len(preds)) :
    ids.append(counter)
    counter += 1
final = pd.DataFrame(ids)
final['id'] = ids
final['Outcome'] = preds
final = final.drop([0],axis=1)
submission = final.to_csv('./submission.csv',index=False)